## Problem Description

A telecom company needs to build a set of cell towers to provide signal coverage  for the inhabitants of a given city. A number of potential locations where the towers could be built have been identified. The towers have a fixed range, and -due to budget constraints- only a limited number of them can be built. Given these restrictions, the company wishes to provide coverage to the largest percentage of the population possible. To simplify the problem, the company has split the area it wishes to cover into a set of regions, each of which has a known population. The goal is then to choose  which of the potential locations the company should build cell towers on -in order to provide coverage to as many people as possible.

The Cell Tower Coverage Problem is an instance of the Maximal Covering Location Problem [1]. It is also related to the Set Cover Problem. Set covering problems occur in many different fields, and very important applications come from the airlines industry. For example, Crew Scheduling and Tail Assignment Problem [2].

## Model Formulation

### Sets and Indices

$i \in T$: Index and set of potential sites to build a tower.

$j \in R$: Index and set of regions.

$G(T,R,E)$: A bipartite graph defined over the set $T$ of potential sites to build a tower,  the set of regions $R$ that we want to cover, and $E$ is the set of edges, where we have an edge $(i,j) \in E$ if region $j \in R$ can be covered by a tower on location $i \in T$.

### Parameters

$c_{i} \in \mathbb{R}^+$: The cost of setting up a tower at site $i$.

$p_{j} \in \mathbb{N}$: The population at region $j$.

### Decision Variables

$covered_{j} \in \{0, 1 \}$: This variable is equal to 1 if region $j$ is covered; and 0 otherwise.

$build_{i} \in \{0, 1 \}$: This variable is equal to 1 if tower $i$ is built; and 0 otherwise.

### Objective Function(s)

- **Population covered**. We seek to maximize the total population covered by the towers.

\begin{equation}
\text{Max} \quad Z = \sum_{j \in R} p_{j} \cdot covered_{j}
\tag{0}
\end{equation}

### Constraints

- **Coverage**. For each region $j \in R$ ensure that at least one tower that covers a region must be selected.

\begin{equation}
\sum_{(i,j) \in E} build_{i} \geq covered_{j} \quad \forall j \in R
\tag{1}
\end{equation}

- **Budget**. We need to ensure that the total cost of building towers do not exceed the allocated budget.

\begin{equation}
\sum_{i \in T} c_{i} \cdot build_{i} \leq \text{budget}
\tag{2}
\end{equation}

## References

[1] Richard Church and Charles R. Velle. "The Maximal Covering Location Problem". Papers in Regional Science, 1974, vol. 32, issue 1, 101-118.

[2] Tail Assignment Problem. https://www.gurobi.com/case_study/air-france-tail-assignment-optimization/


## 例子
This example considers a bipartite graph for 6 towers and 9 regions. The following table illustrates which regions (columns) are covered by each cell tower site (rows).

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |  --- |
| Tower 0 | 1 | 1 | - | - | - | 1 | - | - |  - |
| Tower 1 | 1 | - | - | - | - | - | - | 1 |  1 |
| Tower 2 | - | - | 1 | 1 | 1 | - | 1 | - |  - |
| Tower 3 | - | - | 1 | - | - | 1 | 1 | - |  - |
| Tower 4 | 1 | - | 1 | - | - | - | 1 | 1 |  1 |
| Tower 5 | - | - | - | 1 | 1 | - | - | - |  1 |

The population at each region is stated in the following table.

| <i></i> | Region 0 | Region 1 | Region 2 | Region 3 | Region 4 | Region 5 | Region 6 | Region 7 | Region 8 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Population | 523 | 690 | 420 | 1010 | 1200 | 850 | 400 | 1008 | 950 |

The cost to build a cell tower at each location site is stated inthe following table.

| <i></i> | Cost (millions of USD) |
| --- | --- |
| Tower 0 | 4.2 |
| Tower 1 | 6.1 |
| Tower 2 | 5.2 |
| Tower 3 | 5.5 |
| Tower 4 | 4.8 |
| Tower 5 | 9.2 | 

The allocated budget is $\$20,000,000$.

In [21]:
import gurobipy as grb
def model(sites,regions):
    m = grb.Model("cell_tower")
    # 定义变量
    build = m.addVars(len(sites), vtype=grb.GRB.BINARY, name="Build")
    is_covered = m.addVars(len(regions), vtype=grb.GRB.BINARY, name="Is_covered")
    # 定义目标函数
    m.setObjective(is_covered.prod(population),grb.GRB.MAXIMIZE)
    # 定义约束条件
    ## 1
    m.addConstrs((grb.quicksum(build[t] for t in sites if r in coverage[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")
    ## 2
    m.addConstr(build.prod(cost) <= budget, name="budget")
    # 求解
    m.optimize() 
    # 结果展示
    for tower in build.keys():
        if (abs(build[tower].x) > 1e-6):
            print(f"\n Build a cell tower at location Tower {tower}.")
    #todo:返回的变量方式
    return m.objVal,m.getAttr("x")

In [22]:
# Parameters
budget = 20
regions, population = grb.multidict({
    0: 523, 1: 690, 2: 420,
    3: 1010, 4: 1200, 5: 850,
    6: 400, 7: 1008, 8: 950
})

sites, coverage, cost = grb.multidict({
    0: [{0,1,5}, 4.2],
    1: [{0,7,8}, 6.1],
    2: [{2,3,4,6}, 5.2],
    3: [{2,5,6}, 5.5],
    4: [{0,2,6,7,8}, 4.8],
    5: [{3,4,8}, 9.2]
})

In [23]:
obj,build= model(sites,regions)

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 10 rows, 15 columns and 36 nonzeros
Model fingerprint: 0xfa0fabb2
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 21 nonzeros
Variable types: 0 continuous, 10 integer (10 binary)

Root relaxation: objective 7.051000e+03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7051.0000000 7051.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.03 seconds
Thread count 

In [27]:
# 评估解决方案
total_population = 0
for region in range(len(regions)):
    total_population += population[region]
coverage = round(100*obj/total_population, 2)
print(f"\n The population coverage associated to the cell towers build plan is: {coverage} %")
# Percentage of budget consumed to build cell towers
total_cost = 0
for tower in range(len(sites)):
    if (abs(build[tower]) > 0.5):
        total_cost += cost[tower]*int(build[tower])
budget_consumption = round(100*total_cost/budget, 2)
print(f"\n The percentage of budget consumed associated to the cell towers build plan is: {budget_consumption} %")


 The population coverage associated to the cell towers build plan is: 100.0 %

 The percentage of budget consumed associated to the cell towers build plan is: 71.0 %
